# Monte Carlo Option Pricing under Heston Model

## Set up parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

# Model Parameters
r = 0.05
kappa = 1.0
theta = 0.2
eta = 0.5
rho = -0.4

# Initial Conditions
S0 = 100.0
V0 = 0.25

# Option Parameters
T = 1.0
K = 100.0
N_T = 52

# Monte Carlo Iterations
M = 10 ** 5

## Option Payoffs

In [2]:
# Put option
def put_payoff(r,T,K,S):
    # Terminal stock price
    S_T = S[-1]
    return np.exp(-r * T) * np.maximum(K - S_T,0)

# Asian option
def asian_payoff(r,T,N_T,S,dt):
    # Terminal stock price
    S_T = S[-1]
    
    # Stock price at T_n = n/52
    weekly_price = np.zeros(N_T)
    for week_ind in range(N_T):
        time = (week_ind + 1) / N_T # At time 1/52,2/52,...
        weekly_price[week_ind] = S[int(np.round(time / dt))]
    return np.exp(-r * T) * np.maximum(np.mean(weekly_price) - S_T,0)

## Simulation when both schemes are Euler scheme

In [4]:
# Main Loop
for stepsize_param in range(1,5): 
    # Fix number of step size N
    N = 52 * (2 ** stepsize_param)
    dt = T / N # Length of time interval
    
    # Store put and asian price samples with list
    put_sample = list()
    asian_sample = list()
    
    # Start counting time
    start = timeit.default_timer()
    
    for _ in range(M):
        # Each Monte Carlo iteration
        # Build up increments of W^0,W^2
        W_0_incr = np.sqrt(dt) * np.random.randn(N + 1)
        W_2_incr = np.sqrt(dt) * np.random.randn(N + 1)
        
        # Build up increments of W^1
        W_1_incr = rho * W_2_incr + np.sqrt(1 - rho ** 2) * W_0_incr
        
        # Simulation
        S = np.zeros(N + 1)
        V = np.zeros(N + 1)
        S[0] = S0
        V[0] = V0
        
        for i in range(1,N + 1):
            # Euler scheme
            S[i] = S[i - 1] * (1 + r * dt + np.sqrt(V[i - 1]) * W_1_incr[i - 1])
            V[i] = np.maximum(V[i - 1] + kappa * (theta - V[i - 1]) * dt + eta * np.sqrt(V[i - 1]) * W_2_incr[i - 1],0)
        
        # Collect samples of option price
        put_sample.append(put_payoff(r,T,K,S))
        asian_sample.append(asian_payoff(r,T,N_T,S,dt))
        
    # Stop counting time
    stop = timeit.default_timer()
        
    # Take average as estimate
    put_est = np.mean(put_sample)
    asian_est = np.mean(asian_sample)
    
    # Compute CI
    # Compute sample standard deviation
    put_stddev = np.std(put_sample)
    asian_stddev = np.std(asian_sample)
    
    # 95% CI
    put_CI_95_lb = put_est - 1.96 * (put_stddev / np.sqrt(M))
    put_CI_95_ub = put_est + 1.96 * (put_stddev / np.sqrt(M))
    asian_CI_95_lb = asian_est - 1.96 * (asian_stddev / np.sqrt(M))
    asian_CI_95_ub = asian_est + 1.96 * (asian_stddev / np.sqrt(M))
    
    # 99% CI
    put_CI_99_lb = put_est - 2.58 * (put_stddev / np.sqrt(M))
    put_CI_99_ub = put_est + 2.58 * (put_stddev / np.sqrt(M))
    asian_CI_99_lb = asian_est - 2.58 * (asian_stddev / np.sqrt(M))
    asian_CI_99_ub = asian_est + 2.58 * (asian_stddev / np.sqrt(M))
    
    # Calculate the time spent in Monte Carlo
    running_time = stop - start
    
    # Report the results
    print('When step size is ' + str(dt) + ', we get the following results under Euler scheme:')
    print('Put option price is: ' + str(put_est))
    print('Put option price 95% confidence interval is: [' + str(put_CI_95_lb) + ' ,' + str(put_CI_95_ub) + ']')
    print('Put option price 99% confidence interval is: [' + str(put_CI_99_lb) + ' ,' + str(put_CI_99_ub) + ']')
    print('Asian option price is: ' + str(asian_est))
    print('Asian option price 95% confidence interval is: [' + str(asian_CI_95_lb) + ' ,' + str(asian_CI_95_ub) + ']')
    print('Asian option price 99% confidence interval is: [' + str(asian_CI_99_lb) + ' ,' + str(asian_CI_99_ub) + ']')
    print('Monte Carlo running time is: ' + str(running_time) + ' seconds.')
    print(' ')

When step size is 0.009615384615384616, we get the following results under Euler scheme:
Put option price is: 15.65086345948744
Put option price 95% confidence interval is: [15.522249088594252 ,15.779477830380626]
Put option price 99% confidence interval is: [15.481564950862735 ,15.820161968112144]
Asian option price is: 8.687755115120611
Asian option price 95% confidence interval is: [8.613231380291404 ,8.762278849949817]
Asian option price 99% confidence interval is: [8.589657545804615 ,8.785852684436607]
Monte Carlo running time is: 39.196413167 seconds.
 
When step size is 0.004807692307692308, we get the following results under Euler scheme:
Put option price is: 15.602293968581154
Put option price 95% confidence interval is: [15.474074782869602 ,15.730513154292707]
Put option price 99% confidence interval is: [15.43351565269554 ,15.771072284466769]
Asian option price is: 8.656106542598414
Asian option price 95% confidence interval is: [8.581762291131621 ,8.730450794065206]
Asian o

## Simulation when using Milstein scheme for $V_t$

In [5]:
# Main Loop
for stepsize_param in range(1,5): 
    # Fix number of step size N
    N = 52 * (2 ** stepsize_param)
    dt = T / N # Length of time interval
    
    # Store put and asian price samples with list
    put_sample = list()
    asian_sample = list()
    
    # Start counting time
    start = timeit.default_timer()
    
    for _ in range(M):
        # Each Monte Carlo iteration
        # Build up increments of W^0,W^2
        W_0_incr = np.sqrt(dt) * np.random.randn(N + 1)
        W_2_incr = np.sqrt(dt) * np.random.randn(N + 1)
        
        # Build up increments of W^1
        W_1_incr = rho * W_2_incr + np.sqrt(1 - rho ** 2) * W_0_incr
        
        # Simulation
        S = np.zeros(N + 1)
        V = np.zeros(N + 1)
        S[0] = S0
        V[0] = V0
        
        for i in range(1,N + 1):
            # Euler scheme
            S[i] = S[i - 1] * (1 + r * dt + np.sqrt(V[i - 1]) * W_1_incr[i - 1])
            
            # Milstein scheme
            V[i] = np.maximum(V[i - 1] + kappa * (theta - V[i - 1]) * dt + eta * np.sqrt(V[i - 1]) * W_2_incr[i - 1]
                              + (eta ** 2) / 2 * (W_2_incr[i - 1] ** 2 - dt) / 2,0) # Correction term
        
        # Collect samples of option price
        put_sample.append(put_payoff(r,T,K,S))
        asian_sample.append(asian_payoff(r,T,N_T,S,dt))
        
    # Stop counting time
    stop = timeit.default_timer()
        
    # Take average as estimate
    put_est = np.mean(put_sample)
    asian_est = np.mean(asian_sample)
    
    # Compute CI
    # Compute sample standard deviation
    put_stddev = np.std(put_sample)
    asian_stddev = np.std(asian_sample)
    
    # 95% CI
    put_CI_95_lb = put_est - 1.96 * (put_stddev / np.sqrt(M))
    put_CI_95_ub = put_est + 1.96 * (put_stddev / np.sqrt(M))
    asian_CI_95_lb = asian_est - 1.96 * (asian_stddev / np.sqrt(M))
    asian_CI_95_ub = asian_est + 1.96 * (asian_stddev / np.sqrt(M))
    
    # 99% CI
    put_CI_99_lb = put_est - 2.58 * (put_stddev / np.sqrt(M))
    put_CI_99_ub = put_est + 2.58 * (put_stddev / np.sqrt(M))
    asian_CI_99_lb = asian_est - 2.58 * (asian_stddev / np.sqrt(M))
    asian_CI_99_ub = asian_est + 2.58 * (asian_stddev / np.sqrt(M))
    
    # Calculate the time spent in Monte Carlo
    running_time = stop - start
    
    # Report the results
    print('When step size is ' + str(dt) + ', we get the following results under Milstein scheme for V:')
    print('Put option price is: ' + str(put_est))
    print('Put option price 95% confidence interval is: [' + str(put_CI_95_lb) + ' ,' + str(put_CI_95_ub) + ']')
    print('Put option price 99% confidence interval is: [' + str(put_CI_99_lb) + ' ,' + str(put_CI_99_ub) + ']')
    print('Asian option price is: ' + str(asian_est))
    print('Asian option price 95% confidence interval is: [' + str(asian_CI_95_lb) + ' ,' + str(asian_CI_95_ub) + ']')
    print('Asian option price 99% confidence interval is: [' + str(asian_CI_99_lb) + ' ,' + str(asian_CI_99_ub) + ']')
    print('Monte Carlo running time is: ' + str(running_time) + ' seconds.')
    print(' ')

When step size is 0.009615384615384616, we get the following results under Milstein scheme for V:
Put option price is: 15.720614478974477
Put option price 95% confidence interval is: [15.592141457740002 ,15.849087500208952]
Put option price 99% confidence interval is: [15.551502032655627 ,15.889726925293328]
Asian option price is: 8.709709337640458
Asian option price 95% confidence interval is: [8.635332635839596 ,8.78408603944132]
Asian option price 99% confidence interval is: [8.611805311800547 ,8.80761336348037]
Monte Carlo running time is: 42.18534162499998 seconds.
 
When step size is 0.004807692307692308, we get the following results under Milstein scheme for V:
Put option price is: 15.626922437081728
Put option price 95% confidence interval is: [15.49890392475914 ,15.754940949404315]
Put option price 99% confidence interval is: [15.458408272901996 ,15.795436601261459]
Asian option price is: 8.683915271829438
Asian option price 95% confidence interval is: [8.60945988468521 ,8.758